# Peak fitting of selected ROIs in diffraction data
Select one or more regions of interest and fit a simple gaussian peak across all frames

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import DanMAX as DM
style = DM.darkMode(style_dic={'figure.figsize':'large'})

### Import data

In [ ]:
# Insert path for the .h5 file - TIP: Use tap for auto-complete
#fname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/SAMPLE/scan-XXXX.h5'
fname = DM.findScan()

# get the azimuthally integrated filename from master file name
aname = DM.getAzintFname(fname)

# read the integrated data
data = DM.getAzintData(aname)
if type(data['q']) != type(None):
    x = data['q']
    Q = True
else:
    x = data['tth']
    Q = False

# read common meta data from the master file
meta = DM.getMetaData(fname)

# reduce time-resolution to speed up initial analysis
rf = 1
start = None 
end =  None
data = DM.reduceDic(data,reduction_factor=rf,start=start,end=end)
meta = DM.reduceDic(meta,reduction_factor=rf,start=start,end=end)

I = data['I']

#meta_dic = DM.getMetaDic(fname) # extended meta data dictionary
t = meta['time'] # relative time stamp in seconds
T = meta['temp'] # temperature in Kelvin (if available, otherwise None)
I0 = meta['I0']  # relative incident beam intensity "I zero"
E = meta['energy'] # X-ray energy in keV

print(f'Effective time-resolution: {np.mean(np.diff(t)):.2f} s')

# normalize the integrated intensity to I0
I = (I.T/I0).T

### Select regions of interest
Defined in the relevant scattering unit

In [ ]:
# define the approximate region of interest in scattering units
#         label  :   roi
peaks = {'peak_1':{'roi' : [3.4,3.6]},
         'peak_2':{'roi' : [4.2,4.5]}}

# plot the region of interest for the average pattern
plt.figure()
plt.title(DM.getScan_id(fname))
plt.plot(x,np.mean(I,axis=0),'k-',label='average pattern')
# loop through all peaks
for peak in peaks:
    roi = peaks[peak]['roi']
    roi = (x>roi[0]) & (x<roi[1])
    plt.plot(x[roi],np.mean(I,axis=0)[roi],'.',label=peak)
plt.ylabel('I [a.u.]')
if Q:
    plt.xlabel(r'Q [$\AA^{-1}$]')
else:
    plt.xlabel(r'2$\theta$ [$\deg$]')
plt.legend()

### Perform single-peak fitting

In [ ]:
# loop through all peaks
for peak in peaks:
    # create region of interest mask
    roi = peaks[peak]['roi']
    roi = (x>roi[0]) & (x<roi[1])
    
    amplitude, position, FWHM, background, I_calc = [],[],[],[],[]
    for i,y in enumerate(I):
        amp,pos,fwhm,bgr,y_calc = DM.singlePeakFit(x[roi],y[roi])
        if np.isnan(amp):
            print(f'The fit of pattern {i+1} did not converge!' )
        amplitude.append(amp)
        position.append(pos)
        FWHM.append(fwhm)
        background.append(bgr)
        I_calc.append(y_calc)
    # convert to numpy arrays and save results in the peaks dictionary
    peaks[peak]['amplitude'] = np.array(amplitude)
    peaks[peak]['position'] = np.array(position)
    peaks[peak]['FWHM'] = np.array(FWHM)
    peaks[peak]['background'] = np.array(background)
    peaks[peak]['I_calc'] = np.array(I_calc)
    peaks[peak]['integral'] = np.array(np.trapz(I[:,roi],x[roi],axis=1))

# compare the average observed peak to the average fitted peak
global_roi = min([min(peaks[peak]['roi']) for peak in peaks]), max([max(peaks[peak]['roi']) for peak in peaks])
roi = (x>global_roi[0]) & (x<global_roi[1])

plt.figure()
plt.title(DM.getScan_id(fname))
plt.plot(x[roi],np.mean(I,axis=0)[roi],'k.',label='average observed peak')
for peak in peaks:
    # create region of interest mask
    roi = peaks[peak]['roi']
    roi = (x>roi[0]) & (x<roi[1])
    I_calc = peaks[peak]['I_calc']
    plt.plot(x[roi],np.nanmean(I_calc,axis=0),label='average fitted peak')
    
plt.ylabel('I [a.u.]')
if Q:
    plt.xlabel(r'Q [$\AA^{-1}$]')
else:
    plt.xlabel(r'2$\theta$ [$\deg$]')
plt.legend()

### Plot fitting results

In [ ]:
fig, axs = plt.subplots(2, 3,sharex=True)
fig.suptitle(DM.getScan_id(fname))
axs = axs.flatten()

keys = ['amplitude', 'position', 'FWHM', 'background','integral']
for peak in peaks:
    for i,key in enumerate(keys):
        axs[i].plot(t,peaks[peak][key],label=peak)
        axs[i].set_xlabel('Time (s)')
        axs[i].set_title(key)
# plot temperature if available
if type(T) != type(None):
    axs[-1].plot(t,T, label='Temperature')
    axs[-1].set_title('temperature')
else:
    axs[-1].plot(t,I0, label='I0')
    axs[-1].set_title('I0')

axs[-1].set_xlabel('Time (s)')
axs[2].legend()
fig.tight_layout()